# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers.Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.


## Chatbot using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## Use Cases

1. **Chatbot (Basic)** - Zero Shot chatbot with a FM model
2. **Chatbot using prompt** - template(Langchain) - Chatbot with some context provided in the prompt template
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions
4. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.

## Langchain framework for building Chatbot with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

Second process is the user request orchestration , interaction,  invoking and returing the results

![Chatbot](./images/chatbot_lang.png)

## Architecture [Context Aware Chatbot]
![4](./images/context-aware-chatbot.png)


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [2]:
%pip install -U --no-cache-dir boto3
%pip install -U --no-cache-dir  \
    "langchain>=0.1.11" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0
%pip install -U --no-cache-dir transformers

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 150.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 362.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 133.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.9/826.9 kB 302.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 236.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 198.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 232.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 134.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import warnings
from io import StringIO
import sys
import textwrap
import os
from typing import Optional

# External Dependencies:
import boto3

warnings.filterwarnings('ignore')
def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))
        

In [4]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [5]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! What would you like to chat about? I'm Claude, an AI assistant created by Anthropic to be
helpful, harmless, and honest.


As you can see above, we used a basic prompt ("Hi there!") with no special formatting. In the next sections we will see how to enrich it
with more detail.

To learn more about how to write prompts for Claude, check [Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

## Chatbot using prompt template (Langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="Hi there!"))

 Hello! Nice to meet you. I'm Claude, an AI assistant created by Anthropic to be helpful, harmless,
and honest. How can I assist you today?


#### New Questions

Model has responded with intial message, let's ask few questions

In [7]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

 Here are a few tips to help you start a new garden:

- Decide what you want to grow - Think about what types of vegetables, herbs, or flowers you'd like
to plant. Consider your space, sunlight, and climate when choosing. Some easy options for beginners
are tomatoes, zucchini, bell peppers, lettuce, carrots, radishes, and marigolds.

- Prepare the soil - Remove any grass or weeds where you plan to put the garden. Turn over and
loosen the soil, mixing in compost or other organic material to enrich it. Good soil preparation
helps plants thrive.

- Make a plan - Sketch out a simple garden plan for where plants will go. Make sure taller plants
don't block sun from shorter ones. Rotate plant families each year to prevent diseases.

- Pick healthy starter plants - Look for stocky vegetable seedlings without yellowing leaves or
flowers. For herbs and flowers, choose young plants with healthy, vigorous growth. Or start some
seeds indoors for transplanting later.

- Plant at the right time - Fo

#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [9]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

 Sure, growing tomatoes would work well with the gardening tips I provided earlier. Here are some
more specific tomato growing tips:

- Choose disease-resistant tomato varieties - Look for varieties labeled VFN or VFNT as these are
resistant to common tomato diseases like verticillium, fusarium, nematodes, and tobacco mosaic
virus. Some good options are Celebrity, Better Boy, Big Beef.

- Prepare the soil - Tomatoes thrive in fertile, well-draining soil. Mix in compost or rotted manure
before planting. They prefer slightly acidic soil with a pH between 6.0-6.8.

- Plant seedlings 1-2 feet apart - Give tomato plants plenty of space to allow air circulation and
prevent disease. Plant them deep by burying much of the stem.

- Use cages or stakes - Support tomatoes with tomato cages or staking to keep the fruit and foliage
off the ground. This prevents disease and makes harvesting easier.

- Water at the base - Water deeply 1-2 inches per week. Avoid wetting leaves which can promote
diseas

#### Finishing this conversation

In [10]:
print_ww(conversation.predict(input="That's all, thank you!"))

 You're very welcome! I'm glad I could provide some helpful tomato growing tips to get your new
garden started off right. It sounds like you've got a good plan in place. Wishing you the best of
luck with your gardening and a wonderful tomato harvest this season! Feel free to reach out if any
other questions come up as you begin prepping your garden and caring for your tomato plants. I'm
always happy to offer more advice. Enjoy!


Claude is still really talkative. Try changing the prompt to make Claude provide shorter answers.

In [11]:
# turn verbose to true to see the full logs and documents
conversation2= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt2 = PromptTemplate.from_template("""

Human: The following is a  conversation between a human and an AI.
The AI is not talkative and provides succint answers only. The AI does not elaborate unless asked. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation2.prompt = claude_prompt2

print_ww(conversation2.predict(input="Hi there!"))

 Hello!


In [12]:
print_ww(conversation2.predict(input="Give me a few tips on how to start a new garden."))

 Here are a few tips for starting a new garden:

- Decide what you want to grow - vegetables, flowers, herbs? Select plants that are suitable for
your climate and space.

- Prepare the soil - remove weeds, mix in compost or fertilizer to enrich the soil. You want nice
loose soil for plant roots.

- Make a plan - sketch out garden beds and paths so you know what to plant where. Consider sunlight
needs for different plants.

- Start seeds indoors - begin seeds like tomatoes and peppers a few weeks before last frost date.
Harden off plants before transplanting.

- Plant hardy crops first - Cool weather crops like spinach, lettuce, onions can be planted early in
soil. Hold off on warm weather plants.

- Water new plants well - Don't let seedlings dry out as they establish roots. Provide 1-2 inches of
water per week for most vegetables.

- Add mulch - Place mulch like wood chips or straw around plants to retain moisture and discourage
weeds.

- Be patient! It takes time for a new garden to 

In [14]:
print_ww(conversation2.predict(input="Cool. Will that work with tomatoes?"))

 Yes, the tips I provided will work well for growing tomatoes in a new garden. Here are some key
points for tomatoes specifically:

- Determine if you want bush or vining tomato varieties. Vining types will need cages or trellises
for support.

- Start tomato seeds indoors 6-8 weeks before your last expected spring frost. Tomatoes need time to
establish before going outside.

- Transplant tomatoes when soil is warm, after hardening off the seedlings. Wait until all danger of
frost has passed.

- Plant tomatoes deeply, burying much of the stem. This encourages more root growth. Space plants
18-24 inches apart.

- Use tomato cages or staking to support vines as the plants grow. This prevents fallen and rotting
fruit.

- Water at the base of plants, avoiding wet leaves. Provide 1-2 inches of water per week.

- Mulch around tomatoes heavily. This retains moisture and reduces weeds.

- Pinch off suckers (branches between main stem and leaves) to focus energy on fruiting.

Let me know if you

In [15]:
print_ww(conversation2.predict(input="That's all, thank you!"))

 You're very welcome! I'm glad I could provide some helpful tips for starting a new vegetable
garden, especially for growing tomatoes. Best of luck with your gardening this season! Feel free to
reach out if you have any other questions as you get your new garden going.


### Interactive session using ipywidgets

The following utility class allows us to interact with Claude in a more natural way. We write out the question in an input box, and get Claude's answer. We can then continue our conversation.

In [16]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Let's start a chat. You can also test the following questions:
1. tell me a joke
2. tell me another joke
3. what was the first joke about
4. can you make another joke on the same topic of the first joke

In [17]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## Chatbot with persona

AI assistant will play the role of a career coach. Role Play Dialogue requires user message to be set in before starting the chat. ConversationBufferMemory is used to pre-populate the dialog

In [18]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="What are the career options in AI?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
</conversation_history>

Here is the human's next reply:
<human_reply>
What are the career options in AI?
</human_reply>

Assistant:


> Finished chain.
 Here are some of the top career options in AI (artificial intelligence):

- Machine Learning Engineer - These engineers develop and implement machine learning algorithms and
models to enable systems and software to improve and learn over time. Skills needed include
programming languages like Python and familiarity with ML frameworks like TensorFlow.

- 

In [19]:
print_ww(conversation.predict(input="What these people really do? Is it fun?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
Human: What are the career options in AI?
AI:  Here are some of the top career options in AI (artificial intelligence):

- Machine Learning Engineer - These engineers develop and implement machine learning algorithms and models to enable systems and software to improve and learn over time. Skills needed include programming languages like Python and familiarity with ML frameworks like TensorFlow.

- Data Scientist - Data scientists analyze and interpret complex data to uncover insights, trends, and opport

##### Let's ask a question that is not specialty of this Persona and the model shouldn't answer that question and give a reason for that

In [20]:
conversation.verbose = False
print_ww(conversation.predict(input="How do I fix my car?"))

 I apologize, but I do not have enough context to provide helpful advice on how to fix your car.
Cars can have many different issues, from mechanical problems to electrical faults, and the solution
would depend on the specific make, model and symptoms you are experiencing. Without more details on
what problem you are facing, I would not want to guess or provide generalized car repair suggestions
that may not apply. I'd recommend consulting a mechanic who can properly diagnose and address the
issues with your car. Let me know if you have any other questions I may be better equipped to
answer!


## Chatbot with Context 
In this use case we will ask the Chatbot to answer question from some external corpus it has likely never seen before. To do this we apply a pattern called RAG (Retrieval Augmented Generation): the idea is to index the corpus in chunks, then look up which sections of the corpus might be relevant to provide an answer by using semantic similarity between the chunks and the question. Finally the most relevant chunks are aggregated and passed as context to the ConversationChain, similar to providing a history.

We will take a csv file and use **Titan Embeddings Model** to create vectors for each line of the csv. This vector is then stored in FAISS, an open source library providing an in-memory vector datastore. When the chatbot is asked a question, we query FAISS with the question and retrieve the text which is semantically closest. This will be our answer. 

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [21]:
from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [22]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_aws = loader.load() #
print(f"Number of documents={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Number of documents after split and chunking={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings
    )

    print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to rag_data/Amazon_SageMaker_FAQs.csv
Number of documents=153
Number of documents after split and chunking=154
vectorstore_faiss_aws: number of elements in the index=154::


#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [23]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=cl_llm))

 Based on the provided context, here are some key points about using R with Amazon SageMaker:

- R is supported in SageMaker through RStudio Workbench and SageMaker Studio notebooks with a
preinstalled R kernel. This allows you to develop R models and scripts and integrate them into the
SageMaker ML workflow.

- Reticulate provides an R interface to the SageMaker Python SDK, enabling you to build, train,
tune, and deploy R models using the SageMaker infrastructure and capabilities.

- SageMaker Studio provides a unified visual interface to work with RStudio IDE and SageMaker
notebooks seamlessly, with automatic syncing of artifacts between environments.

- You can leverage SageMaker Experiments to track and compare R modeling experiments and iterations
in the Studio UI.

- Overall, R is a fully supported language in SageMaker with capabilities to develop, experiment,
and deploy R ML models at scale using SageMaker's managed infrastructure and tools.


Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [24]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

[1.171875, 0.33398438, 0.3125, -0.24316406, 0.60546875, 0.41992188, -0.36132812, -6.580353e-05, 0.3203125, -0.66796875]
﻿What is Amazon SageMaker?: Is R supported with Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Yes,
R is supported with Amazon SageMaker. You can use R within SageMaker notebook instances, which
include a preinstalled R kernel and the reticulate library. Reticulate offers an R interface for the
Amazon SageMaker Python SDK, enabling ML practitioners to build, train, tune, and deploy R models.
----
﻿What is Amazon SageMaker?: What is RStudio on Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.:
RStudio on Amazon SageMaker is the first fully managed RSt

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [25]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [26]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

Let's chat! ask the chatbot some questions about SageMaker, like:
1. What is SageMaker?
2. What is canvas?

In [27]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

Your mileage might vary, but after 2 or 3 questions you will start to get some weird answers. In some cases, even in other languages.
This is happening for the same reasons outlined at the beginning of this notebook: the default langchain prompts are not optimal for Claude. 
In the following cell we are going to set two new prompts: one for the question rephrasing, and one to get the answer from that rephrased question.

In [28]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

1. What is SageMaker?
2. what is canvas?

In [29]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens_to_sample` (eg 1000 or 2000) to get the full answer.

In [32]:
# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 2000})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Provide a lengthy and detailed answer for the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

In [33]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

### In this demo we used Claude LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context